In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score
from helperFunctions import *
import optuna

# MLP 모델 정의
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLP, self).__init__()
        layers = []
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(input_size, hidden_size))
            layers.append(nn.ReLU())
            input_size = hidden_size
        layers.append(nn.Linear(input_size, output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 학습 함수 정의
def train_and_validate(model, optimizer, criterion, X_train, y_train, X_val, y_val, epochs, batch_size):
    for epoch in range(epochs):
        model.train()
        permutation = torch.randperm(X_train.size(0))
        for i in range(0, X_train.size(0), batch_size):
            indices = permutation[i:i+batch_size]
            batch_x, batch_y = X_train[indices], y_train[indices]
            
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # 검증 정확도 계산
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_predictions = torch.argmax(val_outputs, dim=1)
        val_accuracy = accuracy_score(y_val.cpu(), val_predictions.cpu())
        
    return val_accuracy

# Optuna objective function
def objective(trial):
    # 하이퍼파라미터 샘플링
    hidden_size1 = trial.suggest_int('hidden_size1', 50, 200)
    hidden_size2 = trial.suggest_int('hidden_size2', 20, 100)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    batch_size = trial.suggest_int('batch_size', 16, 128)
    epochs = trial.suggest_int('epochs', 5, 20)

    # 데이터셋 준비 및 다항식 특징 생성
    n = 30
    e = 2  # 예시 차수
    X_train, X_val, X_test, y_train, y_val, y_test = preprocess(n=n, validate=True, standardize=True)
    poly = PolynomialFeatures(degree=e)
    X_train_poly = poly.fit_transform(X_train)
    X_val_poly = poly.transform(X_val)
    X_test_poly = poly.transform(X_test)

    # 텐서 변환
    X_train_tensor = torch.tensor(X_train_poly, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
    X_val_tensor = torch.tensor(X_val_poly, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

    # 모델 초기화
    input_size = X_train_poly.shape[1]
    output_size = len(np.unique(y_train))
    model = MLP(input_size=input_size, hidden_sizes=[hidden_size1, hidden_size2], output_size=output_size).to(device)
    
    # 옵티마이저 및 손실함수 정의
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 학습 및 검증 정확도 계산
    val_accuracy = train_and_validate(model, optimizer, criterion, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, epochs, batch_size)
    
    return val_accuracy

# Optuna study 시작
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# 최적 하이퍼파라미터 출력
print("Best hyperparameters:", study.best_params)


[I 2024-11-14 02:16:36,676] A new study created in memory with name: no-name-e1059d4d-dd9d-4baa-9e1c-5ac415b7b1ec
/var/folders/cc/95y4d8f130j8q9tr4sn6xvmr0000gn/T/ipykernel_58716/1746643271.py:56: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
[I 2024-11-14 02:16:38,468] Trial 0 finished with value: 0.4975833333333333 and parameters: {'hidden_size1': 110, 'hidden_size2': 59, 'learning_rate': 0.02683462340578616, 'batch_size': 124, 'epochs': 9}. Best is trial 0 with value: 0.4975833333333333.
/var/folders/cc/95y4d8f130j8q9tr4sn6xvmr0000gn/T/ipykernel_58716/1746643271.py:56: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=

Best hyperparameters: {'hidden_size1': 192, 'hidden_size2': 72, 'learning_rate': 0.0008442858251215278, 'batch_size': 88, 'epochs': 7}


In [ ]:
#N=9

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score
from helperFunctions import preprocess

# 실험할 n 및 e 값 설정
possible_n_vals = [9, 12, 15, 18, 24, 30]
possible_e_vals = [1, 2, 3, 4]

# 성능 결과를 저장할 리스트
val_accuracies = []
test_accuracies = []
feature_counts = []

# MLP 모델 정의
class MLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, num_classes)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 실험 시작
for n in possible_n_vals:
    single_val_accuracies = []
    single_test_accuracies = []
    single_feature_counts = []
    
    for e in possible_e_vals:
        # 데이터셋 로드 및 전처리
        X_train, X_val, X_test, y_train, y_val, y_test = preprocess(n=n, validate=True, standardize=True)

        # 다항식 특징 생성
        poly = PolynomialFeatures(degree=e)
        X_train_poly = poly.fit_transform(X_train)
        X_val_poly = poly.transform(X_val)
        X_test_poly = poly.transform(X_test)

        # 텐서로 변환
        X_train_tensor = torch.tensor(X_train_poly, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
        X_val_tensor = torch.tensor(X_val_poly, dtype=torch.float32).to(device)
        y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)
        X_test_tensor = torch.tensor(X_test_poly, dtype=torch.float32).to(device)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

        # 모델 초기화
        model = MLP(input_size=X_train_poly.shape[1], num_classes=len(np.unique(y_train))).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        # 학습
        epochs = 10
        batch_size = 32
        for epoch in range(epochs):
            model.train()
            permutation = torch.randperm(X_train_tensor.size(0))
            for i in range(0, X_train_tensor.size(0), batch_size):
                indices = permutation[i:i+batch_size]
                batch_x, batch_y = X_train_tensor[indices], y_train_tensor[indices]
                
                optimizer.zero_grad()
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        # 검증 및 테스트 데이터 평가
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_predictions = torch.argmax(val_outputs, dim=1).cpu().numpy()
            val_accuracy = accuracy_score(y_val, val_predictions)
            
            test_outputs = model(X_test_tensor)
            test_predictions = torch.argmax(test_outputs, dim=1).cpu().numpy()
            test_accuracy = accuracy_score(y_test, test_predictions)
        
        # 생성된 특징 수 계산 및 결과 저장
        features = X_train_poly.shape[1]
        single_val_accuracies.append(val_accuracy)
        single_test_accuracies.append(test_accuracy)
        single_feature_counts.append(features)
        
        print(f"n={n}, e={e}, Validation Accuracy={val_accuracy:.4f}, Test Accuracy={test_accuracy:.4f}")

    # 각 n에 대한 성능과 특징 수 저장
    val_accuracies.append(single_val_accuracies)
    test_accuracies.append(single_test_accuracies)
    feature_counts.append(single_feature_counts)

# 최종 결과 출력
print("Validation Accuracies by n and e:", val_accuracies)
print("Test Accuracies by n and e:", test_accuracies)
print("Feature Counts by n and e:", feature_counts)


n=9, e=1, Validation Accuracy=0.9603, Test Accuracy=0.9572
n=9, e=2, Validation Accuracy=0.9597, Test Accuracy=0.9569
n=9, e=3, Validation Accuracy=0.9553, Test Accuracy=0.9539
n=9, e=4, Validation Accuracy=0.9533, Test Accuracy=0.9547
n=12, e=1, Validation Accuracy=0.9490, Test Accuracy=0.9525
n=12, e=2, Validation Accuracy=0.9413, Test Accuracy=0.9390
n=12, e=3, Validation Accuracy=0.9340, Test Accuracy=0.9398
n=12, e=4, Validation Accuracy=0.9306, Test Accuracy=0.9300
n=15, e=1, Validation Accuracy=0.8763, Test Accuracy=0.8768
n=15, e=2, Validation Accuracy=0.5238, Test Accuracy=0.5258
n=15, e=3, Validation Accuracy=0.5635, Test Accuracy=0.5543
n=15, e=4, Validation Accuracy=0.6070, Test Accuracy=0.6103
n=18, e=1, Validation Accuracy=0.7796, Test Accuracy=0.7719
n=18, e=2, Validation Accuracy=0.5143, Test Accuracy=0.5125
n=18, e=3, Validation Accuracy=0.5082, Test Accuracy=0.5089
n=18, e=4, Validation Accuracy=0.5008, Test Accuracy=0.5046
n=24, e=1, Validation Accuracy=0.5024, Test 

In [ ]:
import numpy as np
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score
from helperFunctions import *

"""
parameter tuner

"""

# Define MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLP, self).__init__()
        layers = []
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(input_size, hidden_size))
            layers.append(nn.ReLU())
            input_size = hidden_size
        layers.append(nn.Linear(input_size, output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

def optimize_hyperparameters(n, e):
    # 데이터 전처리
    X_train, X_val, X_test, y_train, y_val, y_test = preprocess(n=n, validate=True, standardize=True)
    poly = PolynomialFeatures(degree=e)
    X_train_poly = poly.fit_transform(X_train)
    X_val_poly = poly.transform(X_val)

    X_train_tensor = torch.tensor(X_train_poly, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_val_tensor = torch.tensor(X_val_poly, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def objective(trial):
        # 하이퍼파라미터 샘플링
        hidden_layer1 = trial.suggest_int('hidden_layer1', 50, 200)
        hidden_layer2 = trial.suggest_int('hidden_layer2', 25, 100)
        lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
        batch_size = trial.suggest_int('batch_size', 16, 128)
        epochs = trial.suggest_int('epochs', 5, 30)

        # 모델 초기화
        model = MLP(input_size=X_train_poly.shape[1], hidden_sizes=[hidden_layer1, hidden_layer2],
                    output_size=len(np.unique(y_train))).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # 학습
        for epoch in range(epochs):
            model.train()
            permutation = torch.randperm(X_train_tensor.size(0))
            for i in range(0, X_train_tensor.size(0), batch_size):
                indices = permutation[i:i+batch_size]
                batch_x, batch_y = X_train_tensor[indices].to(device), y_train_tensor[indices].to(device)

                optimizer.zero_grad()
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        # 검증 데이터 평가
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor.to(device))
            val_predictions = torch.argmax(val_outputs, dim=1)
            val_accuracy = accuracy_score(y_val, val_predictions.cpu().numpy())
        return val_accuracy

    # Optuna 최적화 실행
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=20)

    # 최적의 하이퍼파라미터 반환
    best_params = study.best_params
    print("Best hyperparameters:", best_params)
    return best_params


# preprocess dataset for
n = 30
X_train, X_val, X_test, y_train, y_val, y_test = preprocess(n=n, validate=True, standardize=True)


#degree
possible_e_vals = [1]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for e in possible_e_vals:

    best_params = optimize_hyperparameters(n, e)
    # 다항식 특징 생성
    poly = PolynomialFeatures(degree=e)
    X_train_poly = poly.fit_transform(X_train)
    X_val_poly = poly.transform(X_val)
    X_test_poly = poly.transform(X_test)

    # 텐서로 변환
    X_train_tensor = torch.tensor(X_train_poly, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
    X_val_tensor = torch.tensor(X_val_poly, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)
    X_test_tensor = torch.tensor(X_test_poly, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

    # 모델 초기화
    model = MLP(input_size=X_train_poly.shape[1],
            hidden_sizes=[best_params['hidden_layer1'], best_params['hidden_layer2']],
            output_size=len(np.unique(y_train))).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=best_params['lr'])

    # 학습
    for epoch in range(best_params['epochs']):
        model.train()
        permutation = torch.randperm(X_train_tensor.size(0))
        for i in range(0, X_train_tensor.size(0), best_params['batch_size']):
            indices = permutation[i:i+best_params['batch_size']]
            batch_x, batch_y = X_train_tensor[indices], y_train_tensor[indices]

            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
        # 검증 및 테스트 데이터 평가
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_predictions = torch.argmax(val_outputs, dim=1).cpu().numpy()
            val_accuracy = accuracy_score(y_val, val_predictions)
            
            test_outputs = model(X_test_tensor)
            test_predictions = torch.argmax(test_outputs, dim=1).cpu().numpy()
            test_accuracy = accuracy_score(y_test, test_predictions)
        
        # 생성된 특징 수 계산 및 결과 저장
        
    print(f"n={n}, e={e}, Validation Accuracy={val_accuracy:.4f}, Test Accuracy={test_accuracy:.4f}")
targetAcc = targetPerformance(n)
print(f"Target Accuracy:{targetAcc:.4f}")

[I 2024-11-14 02:40:55,148] A new study created in memory with name: no-name-261beada-ada3-49ab-8d32-c4266e2582d7
/var/folders/cc/95y4d8f130j8q9tr4sn6xvmr0000gn/T/ipykernel_58716/475529609.py:44: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
[I 2024-11-14 02:40:58,095] Trial 0 finished with value: 0.5008333333333334 and parameters: {'hidden_layer1': 64, 'hidden_layer2': 70, 'lr': 0.0001510689264056825, 'batch_size': 125, 'epochs': 23}. Best is trial 0 with value: 0.5008333333333334.
/var/folders/cc/95y4d8f130j8q9tr4sn6xvmr0000gn/T/ipykernel_58716/475529609.py:44: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.sug

KeyboardInterrupt: 